In [ ]:
import requests
import pandas as pd
import math
import base64
from io import StringIO

In [ ]:
# Set up the API URL and credentials
API_URL = 'https://your.api.url.here/remotecontrol'
API_USER = "your_username"
API_PASSWORD = "your_pw"
survey_id = "447576"  # LimeSurvey questionaire ID

In [ ]:
# will setup limsurvey connection, if successful returns session key
def setup_limesurvey():
    auth_data = {
        "method": "get_session_key",
        "params": [
            API_USER,
            API_PASSWORD
        ],
        "id": 1
    }
    response = requests.post(API_URL, json=auth_data)

    # check if connection to server can be established
    if response.status_code == 200:
        response_json = response.json()
        # check if result exists in response_json
        if 'result' in response_json:
            result = response_json['result']
            # check if string is 32 characters long (meaning it is the session key)
            if isinstance(result, str) and len(result) == 32:
                session_key = result
                # Use the session_key for subsequent API requests
                return session_key
            else:
                print("Authentication failed:", response_json['result'])
        else:
            print("Authentication failed:", response_json['error']['message'])
    else:
        print("Connection error:", response.status_code)


In [ ]:
# Setup LimeSurvey connecton and get session key
session_key = setup_limesurvey()


In [ ]:
# json payload for token count -> is used for importing user data
json_payload_token_count = {
	"method": "get_summary",
	"params": [
		session_key,
		survey_id,
		"token_count"
	],
	"id": 1
}

token_count = requests.post(API_URL, json=json_payload_token_count).json()["result"]

In [ ]:

# API Call with more than 100'000 rows throws an error, but relevant survey has more than 230'000 rows
# make API calls in steps of 10'000
df_user = pd.DataFrame()  # Create an empty DataFrame
max = 10000 # cases to import per API call
maxIt = math.ceil(int(token_count) / max) # how many API calls are need to load entire data

for i in range(1, maxIt+1):
    json_payload_list_participants = {
        "method": "list_participants",
        "params": [
            session_key,
            survey_id,
            ((i - 1) * max), # iStart (row index to start import)
            max, #iLimit (how many cases to import per API call)
            False, #bUnused (only unused tokens?)
            [
                "completed", # survey completed
                "attribute_1", # geschlecht
                "attribute_2", # alter
                "attribute_3", # pflegerischeoe (Organisationseinehit)
                "attribute_4", # fachlicheoe (Organisationseinehit)
                "attribute_5", # fachlicheoenamen
                "attribute_6"  # klinik
		    ]
        ],
        "id": 1
    }
    
    result_iterate = requests.post(API_URL, json=json_payload_list_participants).json()["result"]
    df_iterate = pd.json_normalize(result_iterate)
    df_user = pd.concat([df_user, df_iterate])


In [ ]:
# json payload for user responses -> is returned as a base64 string
json_payload_export_responses = {
	"method": "export_responses",
	"params": [
		session_key,
		survey_id,
		"csv", # sDocumentType
        "de", # sLanguageCode
        "all", # sCompletionStatus
        "code", # sHeadingType
        "short" # 
	],
	"id": 1
}

response = requests.post(API_URL, json=json_payload_export_responses).json()["result"]
decoded_data = base64.b64decode(response).decode('utf-8')
data_file = StringIO(decoded_data)
# Read the CSV data into a DataFrame
df_responses = pd.read_csv(data_file, delimiter=';')

In [ ]:
# only keep user that have completed the survey
df_responses_comp = df_responses[~pd.isnull(df_responses['submitdate'])]

In [ ]:
# Merge user data and user responses into one df, only keep rows that have a response
df_complete = df_user.merge(df_responses_comp, on='token', how='right')